<a href="https://colab.research.google.com/github/swaragandhi5/Neural_Network/blob/main/Copy_of_Sentiment_Ananlysis_Test_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import required packages
import os
import pandas as pd
import numpy as np
from tensorflow.keras.utils import get_file
import glob

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Creating dataset with reviews and sentiments as column
def load_testing_data(test_list):
    df_test_data = pd.DataFrame(columns=['Reviews','Feedback'])
    for folder in test_list:
        if folder == 'neg' or folder == 'pos':
            file_list = os.listdir(Path+'/'+folder)
            for subfile in file_list:
                fetch_data = open(Path+'/'+folder+'/'+subfile, encoding="utf8")
                if folder == 'neg':
                    df_test_data = df_test_data.append({'Reviews': fetch_data.read(), 'Feedback': 0}, ignore_index=True)
                else:
                    df_test_data = df_test_data.append({'Reviews': fetch_data.read(), 'Feedback': 1}, ignore_index=True)
                    
                    
    return df_test_data

In [ ]:
#Removing html tags 
def review_to_words(review, string = True):
    print("Removing special characters from training dataset:")
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\*)|(\d+)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    NON_LETTERS = re.compile("[^a-zA-Z]")
    NO_SPACE = ""
    SPACE = " "
    review = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in review]
    review = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in review]
    review = [NON_LETTERS.sub(SPACE, line) for line in review]  
    return review

In [ ]:
if __name__ == "__main__": 
	# 1. load your training data
    Path = 'C:/Users/swara/OneDrive/Documents/aclImdb_v1.tar/aclImdb/test'
    test_list = os.listdir(Path)
    
    test_data_df = load_testing_data(test_list)
    
    model=tf.keras.models.load_model('C:/Users/swara/models/Group44_NLP_model.h5')
    

    #2. Removing stopwords
    stopword = set(stopwords.words("english"))
    test_data_df['Reviews_stopwords'] = test_data_df['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword)]))
    
        
    #3. cleaning
    clean_test_reviews = review_to_words(test_data_df.Reviews_stopwords)
    test_data_df = test_data_df.drop(columns='Reviews')
    test_data_df = test_data_df.drop(columns='Reviews_stopwords')
    test_data_df.insert(0,'Reviews', clean_test_reviews)
    
    
    
    Reviews_clean_test = test_data_df['Reviews'].tolist()
    Feedback_clean_test = test_data_df['Feedback'].tolist()
    
    #4. Adding Tokenizer and padding
    t = keras.preprocessing.text.Tokenizer()
    t.fit_on_texts(Reviews_clean_test)
    Reviews_tokenized_test = t.texts_to_sequences(Reviews_clean_test)
    Reviews_padded_test = keras.preprocessing.sequence.pad_sequences(Reviews_tokenized_test,padding='post',maxlen=1000)


FileNotFoundError: ignored

In [ ]:
scores = model.evaluate(tf.convert_to_tensor(Reviews_padded_test),tf.convert_to_tensor(Feedback_clean_test))
test_accuracy = scores[1]
print('accuracy on testing dataset is:',test_accuracy*100)